# Clustering with GPT-3.5-turbo, GPT-4, and Langchain

In [2]:
import os
import tiktoken
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [3]:
from dotenv import load_dotenv
load_dotenv()
import openai

### Dataset is 1,200 DS related job posting from Indeed

In [3]:
# df = pd.read_csv('./data/data_science_jobs_indeed_usa.csv')

In [4]:
# encoding = tiktoken.get_encoding('cl100k_base')

In [5]:
# df = df.assign(token_count=df['Description'].apply(lambda x: len(encoding.encode(x))))

In [6]:
# print(df.shape)
# df.head()

(1200, 11)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,NaN,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25
2,2,IT Business Intelligence Developer (FT) Remote...,Ballad Health,"Remote in Blountville, TN",3.0,PostedPosted 30+ days ago,NaN,Job Details Apply Save Print this job Email a…,https://www.indeed.com/rc/clk?jk=58612836c63b8...,Job Details\nApply\nSave\nPrint this job\nEmai...,10
3,3,Data Engineer,Longevity Holdings Inc.,"Remote in Minneapolis-Saint Paul, MN",NaN,PostedPosted 3 days ago,"$90,000 - $110,000 a year",Incorporate core data management competencies ...,https://www.indeed.com/company/TwentyFirst/job...,Position: Data Engineer\nLocation: MN\nAs a Da...,29
4,4,Network Administrator/dba developer,WKI Kenworth,"Wichita, KS 67219",NaN,EmployerActive 2 days ago,"$50,000 - $70,000 a year",The Network Administrator provides 2nd level e...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Full Job Description\nThe Network Administrato...,28


In [3]:
# Azure: 
# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
# def get_embedding(text) -> list[float]:
#     text = text.replace("\n", " ")
#     return openai.Embedding.create(input=text, engine=OpenAiEmbedding)["data"][0]["embedding"]

# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
# def get_embedding(text, model="text-embedding-ada-002"):
#    text = text.replace("\n", " ")
#    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [8]:
# df = df.assign(embedding=df['Description'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002')))

In [9]:
# df.to_parquet("job_descriptions_with_embeddings.parquet")

In [1]:
df = pd.read_parquet("job_descriptions_with_embeddings.parquet")

NameError: name 'pd' is not defined

In [5]:
df.shape

(1200, 12)

In [6]:
df.head()

,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ..."
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,..."
2,2,IT Business Intelligence Developer (FT) Remote...,Ballad Health,"Remote in Blountville, TN",3.0,PostedPosted 30+ days ago,None,Job Details Apply Save Print this job Email a…,https://www.indeed.com/rc/clk?jk=58612836c63b8...,Job Details\nApply\nSave\nPrint this job\nEmai...,10,"[-0.021908748894929886, -0.002960818586871028,..."
3,3,Data Engineer,Longevity Holdings Inc.,"Remote in Minneapolis-Saint Paul, MN",NaN,PostedPosted 3 days ago,"$90,000 - $110,000 a year",Incorporate core data management competencies ...,https://www.indeed.com/company/TwentyFirst/job...,Position: Data Engineer\nLocation: MN\nAs a Da...,29,"[-0.017482835799455643, -0.01076465379446745, ..."
4,4,Network Administrator/dba developer,WKI Kenworth,"Wichita, KS 67219",NaN,EmployerActive 2 days ago,"$50,000 - $70,000 a year",The Network Administrator provides 2nd level e...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Full Job Description\nThe Network Administrato...,28,"[0.0011034636991098523, -0.000915585900656879,..."


In [7]:
X = np.vstack(df.embedding.values)

* `n_clusters=6` is arbitrary & just for example

In [8]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=42)
df['kmeans_label'] = kmeans.fit_predict(X)

In [9]:
print(df.shape)
df.head(2)

(1200, 13)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,...",2


In [10]:
df.token_count.describe()

count    1200.000000
mean       26.441667
std         4.249507
min         9.000000
25%        24.000000
50%        26.000000
75%        29.000000
max        45.000000
Name: token_count, dtype: float64

In [11]:
from src_index.token_catcher import Usage

usage = Usage()

In [26]:
import langchain
from langchain.cache import SQLiteCache
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import tiktoken



class ClusterTopicLabeler:
    """
    A class to label clusters of texts with topic titles using the LangChain library.

    Attributes
    ----------
    df : pandas.DataFrame
        The DataFrame containing the clusters and texts.
    cluster_column : str
        The name of the column in df that contains the cluster labels.
    text_column : str
        The name of the column in df that contains the texts.
    max_paragraphs : int
        The maximum number of paragraphs that can be processed at once.
    avg_tokens_per_paragraph : int
        The average number of tokens per paragraph.

    Methods
    -------
    get_prompt():
        Returns the prompt for the LangChain model.
    label_clusters():
        Labels the clusters in df with topic titles.
    """

    def __init__(self, df, cluster_column, text_column, max_tokens=4096):
        """
        Constructs all the necessary attributes for the ClusterTopicLabeler object.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame containing the clusters and texts.
        cluster_column : str
            The name of the column in df that contains the cluster labels.
        text_column : str
            The name of the column in df that contains the texts.
        max_tokens : int
            The maximum number of tokens that can be processed at once by the model.
        """
        self.df = df
        self.cluster_column = cluster_column
        self.text_column = text_column
        self.max_tokens = max_tokens
        
        # Initialize a Tiktoken tokenizer
        self.encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
        
        # Calculate the average number of tokens per paragraph
        self.avg_tokens_per_paragraph = df[text_column].apply(self.count_tokens).mean()

        # Calculate the maximum number of paragraphs that can be processed at once
        self.max_paragraphs = max_tokens // self.avg_tokens_per_paragraph
        
    
    def count_tokens(self, text):
        """Returns the number of tokens in a text string."""
        num_tokens = len(self.encoding.encode(text))
        return num_tokens

    
    def get_prompt(self):
        """
        Returns the prompt for the LangChain model.

        Returns
        -------
        ChatPromptTemplate
            The prompt for the LangChain model.
        """
        system_template = "You're an expert journalist. You're helping me write a compelling topic title for news articles."
        human_template = "Using the following articles, write a topic title that summarizes them.\n\nARTICLES:{articles}\n\nTOPIC TITLE:"

        return ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(system_template),
                HumanMessagePromptTemplate.from_template(human_template),
            ],
            input_variables=["articles"],
        )

    
    def get_text_groups(self, texts):
        """
        Groups the texts based on the token limit.

        Parameters
        ----------
        texts : list
            The list of texts to be grouped.

        Returns
        -------
        list
            The grouped texts.
        """
        # Create a DataFrame for easier manipulation
        df = pd.DataFrame(texts, columns=["text"])

        # Add a new column to the DataFrame representing the token count of each text
        df['tokens'] = df['text'].apply(lambda x: self.count_tokens(x))

        # Calculate cumulative token count
        df['cumulative_tokens'] = df['tokens'].cumsum()

        # Create groups based on the token limit
        df['group'] = (df['cumulative_tokens'] // self.avg_tokens_per_paragraph).astype(int)

        # Ensure that the distribution of the text lengths in each group 
        # matches the overall distribution of the text lengths
        df = df.sample(frac=1).groupby('group').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)

        # Create groups
        groups = df.groupby('group')['text'].apply(list).values

        return groups

    def label_clusters(self):
        """
        Labels the clusters in df with topic titles.

        The generated topic titles are assigned to a new column 'topic_title' in df.
        """
        for c in self.df[self.cluster_column].unique():
            chain = LLMChain(
                llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), prompt=self.get_prompt(), verbose=False
            )
            articles = self.df[self.df[self.cluster_column] == c][self.text_column].tolist()

            # Get the groups of articles based on the token limit
            article_groups = self.get_text_groups(articles)

            # Process the article groups
            for group in article_groups:
                group_str = "\n".join(group)
                result = chain.run(
                    {
                        "articles": group_str,
                    }
                )

                # Assign the generated topic title to each article in the group
                for article in group:
                    self.df.loc[self.df[self.text_column] == article, "topic_title"] = result

In [27]:
df.head(1)

,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4


In [28]:
labeler = ClusterTopicLabeler(df, "kmeans_label", "Description", max_tokens=4096)

In [29]:
from src_index.token_catcher import Usage
usage = Usage()

In [30]:
labeler.label_clusters()

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Tue, 01 Aug 2023 02:13:19 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7efa88145f8747a8-DFW', 'alt-svc': 'h3=":443"; ma=86400'}.


In [36]:
usage.total_tokens()

95942

In [32]:
res_df = labeler.df

In [33]:
res_df.to_parquet("ClusterLabelerTestResults.parquet")

In [2]:
import pandas as pd

res_df = pd.read_parquet("ClusterLabelerTestResults.parquet")

In [2]:
# import langchain
# from langchain.cache import SQLiteCache
# langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


system_template = "You're an expert journalist. You're helping me write a concise topic title for job descriptions."
human_template = "Using the following job description, write a concise tag line.\n\nDESCRIPTIONS:{Description}\n\nTAG LINE:"

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template(human_template),
    ],
    input_variables=["Description"],
)
    
chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), prompt=prompt, verbose=False
)


In [5]:
sample_df = res_df.sample(100)

In [6]:
list_to_chain = sample_df[["Description"]].to_dict('records')

In [7]:
# results = await chain.aapply(list_to_chain)

In [7]:
import asyncio

sem = asyncio.Semaphore(50)

async def delay_wrapper(func, *args, **kwargs):
    async with sem:
        result = await func(*args, **kwargs)
    await asyncio.sleep(2)  # 2 second delay
    return result


In [8]:
result = await delay_wrapper(chain.aapply, list_to_chain)


In [9]:
len(result)

100

In [10]:
result

[{'text': '"Expert in building scalable machine learning systems and pipelines"'},
 {'text': '"Immediate interview and excellent pay rate for Iowa Department of Public Health Center for Acute opportunity"'},
 {'text': '"Data Manager: Leading the Way in Efficient Data Management Systems"'},
 {'text': 'Data Pipeline Manager: Scaling and Enhancing Data Quality for Product Launches'},
 {'text': '"Signal Builder: Utilizing Data and Machine Learning for Macro Asset Forecasting"'},
 {'text': '"Experienced Health Insurance Professional with Business Intelligence and Strong Communication Skills"'},
 {'text': '"Experienced Machine Learning Expert with 10+ years of expertise in clustering, decision tree learning, and artificial neural networks"'},
 {'text': '"Machine Learning Expert: Solving Classification, Clustering, and Information Retrieval Challenges"'},
 {'text': '"Experienced Machine Learning Analyst: Analyzing and Ranking Deep Learning Algorithms for Problem Solving"'},
 {'text': '"Databa

In [15]:
list_to_chain[0]

{'Description': 'Develop hosting platform for machine learning models.\nSupervise the scaling and management of the machine learning modeling ecosystem.'}

In [21]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.cache import SQLiteCache
from langchain.callbacks import get_openai_callback
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

In [22]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [22]:
with get_openai_callback() as cb:
    result = llm.predict("Tell me a joke")
    print(cb)
    print(result)

Tokens Used: 34
	Prompt Tokens: 11
	Completion Tokens: 23
Successful Requests: 1
Total Cost (USD): $6.25e-05
Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!


In [23]:
with get_openai_callback() as cb:
    result = llm.predict("Tell me a joke")
    print(cb)
    print(result)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!


In [34]:
usage.total_tokens()

69269

#### Topics by Cluster Label

In [14]:
df = res_df

In [15]:
# sort clusters by their size, and create a new dataframe with the size and cluster number in sorted order. Maintain other columns
size_and_cluster_number = pd.DataFrame(df.groupby("kmeans_label").size().sort_values(ascending=False))

# print out 10 messages from n cluster
for cluster_number, size in size_and_cluster_number.iterrows():
    print(f"{cluster_number} has {size} messages")
    print(df[df.kmeans_label == cluster_number].sample(4).Description.values)
    print("\n")

4 has 0    281
Name: 4, dtype: int64 messages
['Familiarity with data visualization tools.\nComfortable pulling and manipulating data using SQL.\nParticipates in and oversees the Data Science team in analyzing…'
 "This team's key partners include other product teams, finance, data science, and other teams within data engineering."
 'A storyteller, be able to make sense of data and extrapolate meaningful and actionable insights.\nUse website data to create compelling stories and succinct…'
 'Set data standards and oversee stewardship, ensuring that all products and services have data standards built in from the start.']


3 has 0    274
Name: 3, dtype: int64 messages
['And methodology, and data visualization (PowerBI, Tableau, etc.).\nSalary: * depending on relevant industry experience.\nA Day In The Life of a Data Engineer I*.'
 'A Master’s degree or higher in a relevant field such as computer science, data science, applied statistics, or a quantitative social science field or a bachel

In [63]:
df.groupby('kmeans_label').agg({
    'token_count': ['sum', 'mean']
})

token_count           
                     sum       mean
kmeans_label                       
0                   3641  24.938356
1                   4766  26.477778
2                   3109  28.787037
3                   7529  27.478102
4                   7419  26.402135
5                   5266  24.957346

### Sample questions
We can sample around 4k tokens worth of questions. To do this, we shuffle the questions in each cluster, then cut the first ~4k token chunk from each cluster. This should give us ~50% sample ratio on average

In [17]:
from langchain.text_splitter import CharacterTextSplitter

def create_doc(messages, max_tokens=3800):
    #sample around 125 messages from the cluster
    input_doc = '\n\n'.join(messages)

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=max_tokens, chunk_overlap=0, separator="\n\n")
    # Sanity check
    split_texts = text_splitter.split_text(input_doc)
    
    # text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=16000,chunk_overlap=0,separator="\n\n")
    return split_texts[0]


# for clusters, create docs and save in list
docs = []
for cluster_number, size in size_and_cluster_number.iterrows():
    print(f"{cluster_number} has {size} messages")
    Description = df[df.kmeans_label == cluster_number].Description.values
    doc = create_doc(Description)
    docs.append(doc)

4 has 0    281
Name: 4, dtype: int64 messages
3 has 0    274
Name: 3, dtype: int64 messages
5 has 0    211
Name: 5, dtype: int64 messages
1 has 0    180
Name: 1, dtype: int64 messages
0 has 0    146
Name: 0, dtype: int64 messages
2 has 0    108
Name: 2, dtype: int64 messages


### Set up Map Reduce

In [18]:
from langchain import PromptTemplate


map_template_string = """The following is a list of texts that have been clustered using K-Means.
{questions}

Based on this list of questions, please do 3 things: 
(1) Identify the main theme
(2) Give a list of 3 to 5 main sub-themes
(3) Give a representitive example text in each sub-theme
(4) Estimate the proportion of questions that fall into each theme
(5) Share observations on outliers, i.e. questions that do not seem to belong

Helpful Answer:"""

MAP_PROMPT = PromptTemplate(input_variables=["questions"], template=map_template_string)

In [19]:
def run_map(input_doc):

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)
   
    return map_llm_chain.run(questions=input_doc)

* Single cluster test with GPT-3.5-turbo

In [23]:
from src_index.token_catcher import Usage
usage = Usage()

In [26]:
with get_openai_callback() as cb:
    gpt_35_test_result = run_map(docs[0])
    print(cb)
    print(gpt_35_test_result)
    

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example te

In [25]:
usage.total_tokens()

3880

### GPT-4 Map Reduce

In [112]:
gpt4_results = []

for input_doc in docs:
    result=run_map(input_doc)
    gpt4_results.append(result)

In [52]:
usage.total_tokens()

23889

In [114]:
#print results
for i, result in enumerate(gpt4_results):

    #print size of cluster
    print(f"Cluster {i} has {size_and_cluster_number.iloc[i].values[0]} messages: \n")

    #print the summary
    print(result)

    #print separator
    print("\n===============================================================\n\n")

Cluster 0 has 281 messages: 

Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example text: Define a clear data science and analytics roadmap.
   - Proportion:

In [27]:
usage = Usage()

### GPT-3.5-Turbo Map Reduce

In [28]:
gpt35_results = []
with get_openai_callback() as cb:
    for input_doc in docs:
        result=run_map(input_doc)
        gpt35_results.append(result)
        print(cb)
        print(result)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example te

In [29]:
usage.total_tokens()

18945

In [30]:
#print results
for i, result in enumerate(gpt35_results):

    #print size of cluster
    print(f"Cluster {i} has {size_and_cluster_number.iloc[i].values[0]} messages: \n")

    #print the summary
    print(result)

    #print separator
    print("\n===============================================================\n\n")

Cluster 0 has 281 messages: 

Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example text: Define a clear data science and analytics roadmap.
   - Proportion:

### Send all cases through Map Reduce chain

In [31]:
from langchain.text_splitter import CharacterTextSplitter
input_doc = '\n\n'.join(docs)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000,chunk_overlap=0,separator="\n\n")
# Sanity check
len(text_splitter.split_text(input_doc))

6

In [32]:
map_template_string = """The following is a list of texts that have been clustered using K-Means
{questions}

Based on this list of texts, please do 4 things: 
(1) identify the main themes 
(2) give a represntitive example question in each theme
(3) estimate the proportion of questions that fall into each theme
(4) share observations on outliers, i.e. questions that do not seem to belong

Helpful Answer:"""

reduce_template_string = template = """The following is a list of summaries for questions entered into a Q+A system:
{question_summaries}

Take these and distill it into a final, consolidated list with: 
(1) the main question themes 
(2) two represntitive example questions in each theme
(3) estimate the proportion of questions that fall into each theme
(4) a summary of outliers

Helpful Answer:"""
MAP_PROMPT = PromptTemplate(input_variables=["questions"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["question_summaries"], template=reduce_template_string)

In [33]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import (
                StuffDocumentsChain,
                LLMChain,
                ReduceDocumentsChain,
                MapReduceDocumentsChain,
            )


def run_map_reduce(input_doc, model, MAP_PROMPT, REDUCE_PROMPT):
    
    llm = ChatOpenAI(model_name=model, temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name=model, temperature=0)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    # Takes a list of documents and combines them into a single string
    combine_documents_chain = StuffDocumentsChain(
            llm_chain=reduce_llm_chain,
            document_variable_name="question_summaries")
    
    # Combines and iteravely reduces the mapped documents 
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `combine_documents_chain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000)

    # Combining documents by mapping a chain over them, then combining results
    combine_documents = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_llm_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="questions",
        # Return the results of the map steps in the output
        ### Bug: this currently does not work ###
        return_intermediate_steps=False)
        
    # Define Map=Reduce
    map_reduce = MapReduceChain(
        # Chain to combine documents
        combine_documents_chain=combine_documents,
        # Splitter to use for initial split
        text_splitter=text_splitter)
    
    return map_reduce.run(input_text=input_doc)

In [34]:
len(input_doc)

117588

In [35]:
len(docs[0])

20346

In [36]:
usage = Usage()

In [37]:
map_reduce_gpt4_res = []

with get_openai_callback() as cb:
    for i in range(len(docs)):
        result = run_map_reduce(docs[i], "gpt-4",MAP_PROMPT, REDUCE_PROMPT)
        map_reduce_gpt4_res.append(result)
        print(cb)
        print(f"\nCluster: {i}:")
        print(result)
        print("\n===============================================================\n\n")

Tokens Used: 5059
	Prompt Tokens: 4146
	Completion Tokens: 913
Successful Requests: 2
Total Cost (USD): $0.17915999999999999

Cluster: 0:
(1) Main Question Themes:
   - Data Science Strategy and Management
   - Data Analysis and Insights
   - Data Engineering and Infrastructure
   - Collaboration and Teamwork
   - Data Governance and Security

(2) Representative Example Questions:
   - Data Science Strategy and Management: 
     1. "How do you define and execute a data science strategy that ensures our organization's success?"
     2. "What are the key elements in setting up a successful data science workflow?"
   - Data Analysis and Insights: 
     1. "How do you analyze product data to identify problems and ideate product features?"
     2. "What methods do you use to turn raw data into actionable insights?"
   - Data Engineering and Infrastructure: 
     1. "How do you manage and scale data pipelines from internal and external data sources?"
     2. "What are the best practices for 

In [38]:
usage.total_tokens()

29513

#### GPT-3.5-turbo

In [39]:
usage = Usage()

In [40]:
map_reduce_gpt35_res = []

with get_openai_callback() as cb:
    for i in range(len(docs)):
        result = run_map_reduce(docs[i], "gpt-3.5-turbo", MAP_PROMPT, REDUCE_PROMPT)
        map_reduce_gpt35_res.append(result)
        print(cb)
        print(f"\nCluster: {i}:")
        print(result)
        print("\n===============================================================\n\n")

Tokens Used: 4575
	Prompt Tokens: 3930
	Completion Tokens: 645
Successful Requests: 2
Total Cost (USD): $0.0071849999999999995

Cluster: 0:
(1) Main Question Themes:
- Data Science Strategy and Leadership
- Data Management and Quality
- Collaboration and Teamwork
- Data Analysis and Insights
- Data Infrastructure and Tools

(2) Representative Example Questions:
- Data Science Strategy and Leadership:
  - "How can we define and execute a data science and analytics roadmap?"
  - "What are the best practices for building a data-driven culture within an organization?"

- Data Management and Quality:
  - "How can we ensure data governance, data security, and data quality?"
  - "What are the key steps in data cleaning and preprocessing to improve data quality?"

- Collaboration and Teamwork:
  - "How can we collaborate with other teams to turn data into critical information and knowledge?"
  - "What are effective strategies for fostering cross-functional collaboration in data science project

In [41]:
usage.total_tokens()

27171

### GPT-3.5-turbo

In [43]:
from langchain.text_splitter import CharacterTextSplitter
input_doc = '\n\n'.join(docs)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000,chunk_overlap=0,separator="\n\n")
# Sanity check
len(text_splitter.split_text(input_doc))

6

### Targeted Topic Test

In [44]:
reduce_template_string = template = """The following is a list of summaries:
{question_summaries}

Take these and distill it into a final, consolidated list with: 
(1) the top 5 texts related to AI and ML applications.
(2) estimate the proportion of each question

Helpful Answer:"""
REDUCE_PROMPT = PromptTemplate(input_variables=["question_summaries"], template=reduce_template_string)

#### GPT-4

In [45]:
with get_openai_callback() as cb:
    result = run_map_reduce(docs[3], "gpt-4", MAP_PROMPT, REDUCE_PROMPT)
    print(cb)
    print(result)

Tokens Used: 717
	Prompt Tokens: 482
	Completion Tokens: 235
Successful Requests: 1
Total Cost (USD): $0.028559999999999995
(1) Top 5 Texts Related to AI and ML Applications:
   - "What experience do you have in developing and deploying machine learning models for ad recommendation?"
   - "Can you provide examples of your proficiency with data mining and machine learning?"
   - "What is your understanding of various advanced analytical and machine learning methods?"
   - "Do you have experience with machine learning frameworks such as PyTorch or TensorFlow/Keras?"
   - "Can you describe your industry or research experience in data analytics, machine learning, and software development?"

(2) Estimated Proportion of Each Question:
   - "What experience do you have in developing and deploying machine learning models for ad recommendation?" - 30%
   - "Can you provide examples of your proficiency with data mining and machine learning?" - 20%
   - "What is your understanding of various adva

### GPT3.5 Turbo

In [46]:
usage = Usage()

In [47]:
with get_openai_callback() as cb:
    result = run_map_reduce(docs[3], "gpt-3.5-turbo", MAP_PROMPT, REDUCE_PROMPT)
    print(cb)
    print(result)

Tokens Used: 610
	Prompt Tokens: 509
	Completion Tokens: 101
Successful Requests: 1
Total Cost (USD): $0.0009655000000000001
(1) The top 5 texts related to AI and ML applications are:
- Strategic Communications
- Ad Recommendation
- Natural Language Processing
- Computer Vision
- Image Processing

(2) The estimated proportion of each question is:
- Machine Learning Expertise: 30%
- Data Science and Analytics: 25%
- AI and ML Applications: 30%
- Machine Learning Infrastructure: 15%
- Outliers: 0% (since they do not fit into any specific theme)


In [48]:
usage.total_tokens()

610